In [169]:
filepath = '/home/jupyter/Devjam/audio/'     #Input audio file path
output_filepath = '/home/jupyter/Devjam/' #Final transcript path
bucketname = "devjam_vg" #Name of the bucket created in the step before

# Import libraries
from pydub import AudioSegment
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import wave
from google.cloud import storage
import gcsfs
fs = gcsfs.GCSFileSystem(project='tactile-alloy-266213')
from os.path import dirname, join as pjoin
from scipy.io import wavfile
import scipy.io


In [2]:
!pip install pydub

In [104]:
def mp3_to_wav(audio_file_name):
    if audio_file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(audio_file_name)
        audio_file_name = audio_file_name.split('.')[0] + '.wav'
        sound.export(audio_file_name, format="wav")

In [162]:
def stereo_to_mono(audio_file_name):
    if audio_file_name.split('.')[1] == 'wav':
        sound = AudioSegment.from_wav(audio_file_name)
        sound = sound.set_channels(1)
        sound.export(audio_file_name, format="wav")

In [155]:
def frame_rate_channel(audio_file_name):
    wav_fname = pjoin(filepath, 'VOXTAB_Medical_audio.wav')
    samplerate, data = wavfile.read(wav_fname)
    channels=int(data.shape[1])
    return channels

In [156]:
def frame_rate_channel1(audio_file_name):
    
    #for audio_file in os.listdir(filepath):
    wav_fname = pjoin(filepath, 'VOXTAB_Medical_audio.wav')
    samplerate, data = wavfile.read(wav_fname)
    frame_rate=int(data.shape[0])
    return frame_rate

In [108]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [109]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [171]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    print(audio_file_name)
    print(filepath)
    mp3_to_wav(file_name)
    wav_fname = pjoin(filepath, 'VOXTAB_Medical_audio.wav')

    # The name of the audio file to transcribe
    
    channels = frame_rate_channel(file_name)
    print(channels)
    frame_rate = frame_rate_channel1(file_name)
    print(frame_rate)
    
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = bucketname
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    sample_rate_hertz= 44100,
    language_code='en-US')

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

In [49]:
def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close()

In [172]:
if __name__ == "__main__":
    for audio_file_name in os.listdir(filepath):
        transcript = google_transcribe(audio_file_name)
        transcript_filename = audio_file_name.split('.')[0] + '.txt'
        write_transcripts(transcript_filename,transcript)

VOXTAB_Medical_audio.mp3
/home/jupyter/Devjam/audio/
2
5916672
VOXTAB_Medical_audio.wav
/home/jupyter/Devjam/audio/
2
5916672


In [146]:
os.getcwd()

'/home/jupyter/Devjam'

In [147]:
currentDirectory = os.getcwd()
print(currentDirectory)
    

/home/jupyter/Devjam


In [76]:
from os.path import dirname, join as pjoin
from scipy.io import wavfile
import scipy.io

In [85]:
filepath = '/home/jupyter/Devjam/audio/'
wav_fname = pjoin(filepath, 'VOXTAB_Medical_audio.wav')

In [139]:
samplerate, data = wavfile.read(wav_fname)
channels=int(data.shape[0])
print(channels)


5916672


In [140]:
channels.datatype

AttributeError: 'int' object has no attribute 'datatype'